## Sprint 1

In [2]:
DATA_FOLDER = "actions"

## Exploratory Data Analysis


### What is contained in the name of each file?

The name of the files all have the following structure: `action-X-group-Y-rec-Z.npy`.
- X defines the type of exercise that was performed
 - 0: Jumping jack
 - 1: Squat
 - 2: In-place running
 - 3: Side lunge
 - 4: Boxing
 - 5: Overhead press
 - 6: Bicep curl
- Y defines the group that performed these exercises
- Z defines the person of that group.

The .npy format is the standard binary file format in NumPy for persisting a single arbitrary NumPy array on disk.

  

### How can you filter data by action, or by group?

In [ ]:
import glob

#for actions
filtered_actions = glob.glob(f'{DATA_FOLDER}/*action-[1-2]*.npy')

#for groups
filtered_groups = glob.glob(f'{DATA_FOLDER}/*group-[1-7]*.npy')

Above, it was illustrated how to filter files using standard Regex code.

`*`: this is meant to match any number of characters, including none. This will match any string of characters before the word "action-" or "group-".

`[1-2]`: This part inside square brackets is a character range. It matches a single character that can be either 1 or 2.

A function was made to do this:

In [4]:
def filter_files_by_action_or_group(directory, action_start,action_end, groups_start, groups_end):
    filtered=glob.glob(f"{directory}/*action-[{action_start}-{action_end}]*group-[{groups_start}-{groups_end}]*.npy")
    return filtered

### Is the data balanced?

In [ ]:
filter_files_by_action_or_group(DATA_FOLDER, 0, 6, 0, 9)

### What are the ranges for each feature?

### Do all (values for all) features make sense? Are there any outliers?